In [ ]:
import os
import cv2
import numpy as np
from paddleocr import PaddleOCR
import paddle
import time


print(f"Using GPU: {paddle.is_compiled_with_cuda()}")  # This will still show if the system supports GPU, but we won't use it

# Initialize PaddleOCR with CPU (use_gpu=False)
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=False)

# Video input/output paths
input_video_path = "/content/OCR-Sample.mp4"
output_video_path = "output_video_cpu.mp4"

# Capture video from file
cap = cv2.VideoCapture(input_video_path)

# Get video properties (FPS, width, height, frame count)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Input video FPS: {fps}")
print(f"Video resolution: {width}x{height}")
print(f"Total frames: {frame_count}")

# Define codec and create VideoWriter object for output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Variables for tracking frame number, accuracies, and FPS
frame_num = 0
accuracies = []
fps_values = []

start_time = time.time()

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR frame to RGB for PaddleOCR
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Run OCR on the current frame (CPU version)
    result = ocr.ocr(img_rgb)
    total_confidence = 0
    num_words = 0

    if result is not None:
        for line in result:
            if line:
                for word_info in line:
                    text = word_info[-1][0]  # Extract detected text
                    confidence = word_info[-1][1]  # Extract confidence score
                    total_confidence += confidence
                    num_words += 1

                    bbox = word_info[0]

                    # Convert bbox points to integer and draw them on the frame
                    bbox = [(int(point[0]), int(point[1])) for point in bbox]
                    cv2.polylines(frame, [np.array(bbox)], True, (0, 255, 0), 2)

                    # Put the detected text on the frame
                    cv2.putText(frame, text, bbox[0], cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    # Write processed frame to output video
    out.write(frame)

    # Calculate average accuracy for this frame
    if num_words > 0:
        accuracy = total_confidence / num_words
    else:
        accuracy = 0

    accuracies.append(accuracy)

    # Track frame number and calculate FPS
    frame_num += 1
    elapsed_time = time.time() - start_time
    current_fps = frame_num / elapsed_time
    fps_values.append(current_fps)

    # Print progress
    print(f"Processing frame {frame_num}/{frame_count}, Accuracy: {accuracy:.2f}, FPS: {current_fps:.2f}")

end_time = time.time()

# Calculate total time and average FPS
total_time = end_time - start_time
average_fps = frame_num / total_time

print(f"Total time taken for processing: {total_time:.2f} seconds")
print(f"Average FPS during processing: {average_fps:.2f}")

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processing completed and saved to", output_video_path)


Using GPU: False
[2024/10/06 14:34:23] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text

In [ ]:

import os
import cv2
import numpy as np
from paddleocr import PaddleOCR
import paddle
import time


print(f"Using GPU: {paddle.is_compiled_with_cuda()}")

ocr = PaddleOCR(use_angle_cls=True, lang='en',use_gpu = True)



input_video_path = "/content/OCR-Sample.mp4"
output_video_path = "output_video_gpu.mp4"


cap = cv2.VideoCapture(input_video_path)


fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Input video FPS: {fps}")
print(f"Video resolution: {width}x{height}")
print(f"Total frames: {frame_count}")


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_num = 0
accuracies = []
fps_values = []

start_time = time.time()


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break


    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


    result = ocr.ocr(img_rgb)
    total_confidence = 0
    num_words = 0

    if result is not None:
        for line in result:
            if line:
                for word_info in line:
                    text = word_info[-1][0]
                    confidence = word_info[-1][1]
                    total_confidence += confidence
                    num_words += 1

                    bbox = word_info[0]

                    bbox = [(int(point[0]), int(point[1])) for point in bbox]
                    cv2.polylines(frame, [np.array(bbox)], True, (0, 255, 0), 2)


                    cv2.putText(frame, text, bbox[0], cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)

    out.write(frame)

    if num_words > 0:
        accuracy = total_confidence / num_words
    else:
        accuracy = 0

    accuracies.append(accuracy)


    frame_num += 1
    elapsed_time = time.time() - start_time
    current_fps = frame_num / elapsed_time
    fps_values.append(current_fps)

    print(f"Processing frame {frame_num}/{frame_count}, Accuracy: {accuracy:.2f}, FPS: {current_fps:.2f}")

end_time = time.time()


total_time = end_time - start_time
average_fps = frame_num / total_time

print(f"Total time taken for processing: {total_time:.2f} seconds")
print(f"Average FPS during processing: {average_fps:.2f}")


cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processing completed and saved to", output_video_path)


Using GPU: False
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:00<00:00, 4.76MiB/s]


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:01<00:00, 9.86MiB/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:00<00:00, 3.16MiB/s]

[2024/10/06 14:57:47] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

Input video FPS: 30
Video resolution: 852x480
Total frames: 461
[2024/10/06 14:57:49] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.3895895481109619
[2024/10/06 14:57:49] ppocr DEBUG: cls num  : 10, elapsed : 0.07883095741271973
[2024/10/06 14:57:49] ppocr DEBUG: rec_res num  : 10, elapsed : 0.6379799842834473
Processing frame 1/461, Accuracy: 0.96, FPS: 0.85
[2024/10/06 14:57:50] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.1621387004852295
[2024/10/06 14:57:50] ppocr DEBUG: cls num  : 10, elapsed : 0.035172224044799805
[2024/10/06 14:57:50] ppocr DEBUG: rec_res num  : 10, elapsed : 0.49924731254577637
Processing frame 2/461, Accuracy: 0.96, FPS: 1.05
[2024/10/06 14:57:50] ppocr DEBUG: dt_boxes num : 10, elapsed : 0.1498415470123291
[2024/10/06 14:57:50] ppocr DEBUG: cls num  : 10, elapsed : 0.03625655174255371
[2024/10/06 14:57:51] ppocr DEBUG: rec_res num  : 10, elapsed : 0.5440883636474609
Processing frame 3/461, Accuracy: 0.96, FPS: 1.13
[2024/10/06 14:57:51] ppocr DEBUG: dt_boxes nu